**NNFL Course Project** <br>
**Paper** :  Deep Convolutional Neural Networks for Sentiment Analysis of Short Texts (Paper ID: 211)<br>
**Members**<br>
Indraneel Ghosh 2016B1A70938P<br>
Mohit Kriplani 2016B1A70870P<br>
Himanshu Agarwal 2016B3A30570P <br>

**Guidelines:**
1.   Use Tensorflow, PyTorch or Keras to implement the given CharSCNN architecture given in the paper.

2.  For training: either use STS or SSTb. No need to compare with other baseline models, as specified in the paper.
3.  Report accuracy of your model using pre-trained word embeddings.

**Dataset Link**: http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

References for Keras:
https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456

https://www.kaggle.com/coder98/blood-cell-subtype-identification-cudnn-cnn-rnn

Note: Modify all functions using Keras as a framework

Loading Files to Cloud(Do not run again)

File upload Testing

https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

Code starts

In [0]:
import os
import sys
import random
import math
import subprocess
from glob import glob
from collections import OrderedDict
import numpy as np
import nltk
from skimage import measure
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import scipy.ndimage as ndi
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, Reshape, LSTM, CuDNNLSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from collections import OrderedDict

In [0]:
def read(filename):
    col_names=['A1','A2','A3','A4','A5','A6']
    dat=pd.read_csv(filename,names=col_names,header=None)
    lines=dat['A1'].to_numpy()+dat['A6'].to_numpy()
    #Mohit Kriplani
    words_map = {}
    word_cnt = 0
    
    k_wrd = 5 #check

    y = [] 
    x_wrd = []

    max_sen_len, num_sent = 0, 20000 # Max sentence length and number of sentences
    # f = open(input, 'r')
    # lines = f.readlines()
    
    for line in lines[:num_sent]:
        words = line[:-1].split()
        tokens = words[1:]
        y.append(int(float(words[0])))
        max_sen_len = max(max_sen_len,len(tokens))
        for token in tokens:
            if token not in words_map:
                words_map[token] = word_cnt
                word_cnt += 1
    
    for line in lines[:num_sent]:
        words = line[:-1].split()
        tokens = words[1:]
        word_mat = [0] * (max_sen_len+k_wrd-1)

        for i in xrange(len(tokens)):
            word_mat[(k_wrd/2)+i] = words_map[tokens[i]]
        x_wrd.append(word_mat)
    max_sen_len += k_wrd-1
    # y: 1 or 0 (i.e., positive or negative)
    data = (num_sent, word_cnt, max_sen_len, k_wrd, x_wrd, y)
    return data

**Search for Equivalent library functions before you start writing code**


In [0]:
class EmbedIDLayer(object):
    """
    Efficient linear function for one-hot input.
    """
    #Kriplani
    def __init__(
        self,
        rng,
        input=None,
        n_input=None,
        n_output=None,
        W=None,
    ):
        if input is None:
            input = T.imatrix('x')

        if W is None:
            W_values = np.asarray(
                rng.uniform(low=-np.sqrt(6.0/(n_input+n_output)),
                            high=np.sqrt(6.0/(n_input+n_output)),
                            size=(n_input, n_output)),
                dtype=theano.config.floatX)

            # W_values[0,0] = 0
            W_tmp = theano.shared(value=W_values, name='W', borrow=True)
        else:
            W_values = W.astype(theano.config.floatX)
            W_tmp = theano.shared(value=W_values, name='W', borrow=True)

        self.W = W_tmp
        self.output = self.W[input]
        self.params = [self.W]


Fully connected layer

In [0]:
class FullyConnectedLayer(object):
    def __init__(
        self,
        rng,
        input=None, 
        n_input=784,
        n_output=10,
        activation=None,
        W=None,
        b=None
    ):

        self.input = input

        if W is None:
            W_values = np.asarray(
                rng.uniform(low=-np.sqrt(6.0/(n_input+n_output)),
                            high=np.sqrt(6.0/(n_input+n_output)),
                            size=(n_input, n_output)),
                dtype=theano.config.floatX)
            if activation == sigmoid:
                W_values *= 4.0
            W = theano.shared(value=W_values, name='W', borrow=True)

        if b is None:
            b_values = np.zeros((n_output,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)

        self.W = W
        self.b = b

        linear_output = T.dot(input, self.W) + self.b

        if activation is None:
            self.output = linear_output
        else:
            self.output = activation(linear_output)

        self.params = [self.W, self.b]

In [0]:
class ConvolutionalLayer(object):
  #Indraneel
    def __init__(
        self,
        rng,
        input,
        filter_shape=None,
        image_shape=None,
        activation=relu
    ):
        self.input = input
        self.rng = rng

        """filter shape = n_feature_map, in channel, width, height"""
        """channel * width * height"""
        fan_in = np.prod(filter_shape[1:]) # 1*2*3
        """feature map """
        fan_out = filter_shape[0] * np.prod(filter_shape[2:]) # 0*2*3
        
        W_bound = np.sqrt(6.0 / (fan_in + fan_out))
        self.W = theano.shared(
            np.asarray(
                self.rng.uniform(
                    low = -W_bound,
                    high = W_bound,
                    size = filter_shape
                ),
                dtype = theano.config.floatX
            ),
            borrow = True
        )

        b_values = np.zeros((filter_shape[0],),
                            dtype=theano.config.floatX)
        self.b = theano.shared(b_values, borrow=True)

        conv_out = conv.conv2d(
            input=self.input,
            filters=self.W,
            filter_shape=filter_shape,
            image_shape=image_shape
        )

        self.output = activation(conv_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        self.params = [self.W, self.b]

Maxpooling exists

In [0]:
class MaxPoolingLayer(object):
  #Indraneel
    def __init__(self, input, poolsize=(2,2)):
        pooled_out = downsample.pool_2d(
            input=input,
            ws=poolsize,
            ignore_border=True
        )
        self.output = pooled_out

Keras should have an in-built optimizer function. Please verify this.(Indraneel)[All optimizzers exist]

In [0]:
class Optimizer(object):
    def __init__(self, params=None):
        if params is None:
            return NotImplementedError()
        self.params = params

    def updates(self, loss=None):
        if loss is None:
            return NotImplementedError()

        self.updates = OrderedDict()
        self.gparams = [T.grad(loss, param) for param in self.params]

# SGD exists
class SGD(Optimizer):
    def __init__(self, learning_rate=0.01, params=None):
        super(SGD, self).__init__(params=params)
        self.learning_rate = 0.01

    def updates(self, loss=None):
        super(SGD, self).updates(loss=loss)

        for param, gparam in zip(self.params, self.gparams):
            self.updates[param] = param - self.learning_rate * gparam

        return self.updates
#SGD Momentum exists
class MomentumSGD(Optimizer):
    def __init__(self, learning_rate=0.01, momentum=0.9, params=None):
        super(MomentumSGD, self).__init__(params=params)
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.vs = [build_shared_zeros(t.shape.eval(), 'v') for t in self.params]

    def updates(self, loss=None):
        super(MomentumSGD, self).updates(loss=loss)

        for v, param, gparam in zip(self.vs, self.params, self.gparams):
            _v = v * self.momentum
            _v = _v - self.learning_rate * gparam
            self.updates[param] = param + _v
            self.updates[v] = _v

        return self.updates
#Adagrad exists
class AdaGrad(Optimizer):
    def __init__(self, learning_rate=0.01, eps=1e-6, params=None):
        super(AdaGrad, self).__init__(params=params)

        self.learning_rate = learning_rate
        self.eps = eps
        self.accugrads = [build_shared_zeros(t.shape.eval(),'accugrad') for t in self.params]

    def updates(self, loss=None):
        super(AdaGrad, self).updates(loss=loss)

        for accugrad, param, gparam\
        in zip(self.accugrads, self.params, self.gparams):
            agrad = accugrad + gparam * gparam
            dx = - (self.learning_rate / T.sqrt(agrad + self.eps)) * gparam
            self.updates[param] = param + dx
            self.updates[accugrad] = agrad

        return self.updates
#RMSprop exists
class RMSprop(Optimizer):
    def __init__(self, learning_rate=0.001, alpha=0.99, eps=1e-8, params=None):
        super(RMSprop, self).__init__(params=params)

        self.learning_rate = learning_rate
        self.alpha = alpha
        self.eps = eps

        self.mss = [build_shared_zeros(t.shape.eval(),'ms') for t in self.params]

    def updates(self, loss=None):
        super(RMSprop, self).updates(loss=loss)

        for ms, param, gparam in zip(self.mss, self.params, self.gparams):
            _ms = ms*self.alpha
            _ms += (1 - self.alpha) * gparam * gparam
            self.updates[ms] = _ms
            self.updates[param] = param - self.learning_rate * gparam / T.sqrt(_ms + self.eps)

        return self.updates

# adadelta exists
class AdaDelta(Optimizer):
    def __init__(self, rho=0.95, eps=1e-6, params=None):
        super(AdaDelta, self).__init__(params=params)

        self.rho = rho
        self.eps = eps
        self.accugrads = [build_shared_zeros(t.shape.eval(),'accugrad') for t in self.params]
        self.accudeltas = [build_shared_zeros(t.shape.eval(),'accudelta') for t in self.params]

    def updates(self, loss=None):
        super(AdaDelta, self).updates(loss=loss)

        for accugrad, accudelta, param, gparam\
        in zip(self.accugrads, self.accudeltas, self.params, self.gparams):
            agrad = self.rho * accugrad + (1 - self.rho) * gparam * gparam
            dx = - T.sqrt((accudelta + self.eps)/(agrad + self.eps)) * gparam
            self.updates[accudelta] = (self.rho*accudelta + (1 - self.rho) * dx * dx)
            self.updates[param] = param + dx
            self.updates[accugrad] = agrad

        return self.updates

#Adam exists
class Adam(Optimizer):
    def __init__(self, alpha=0.001, beta1=0.9, beta2=0.999, eps=1e-8, gamma=1-1e-8, params=None):
        super(Adam, self).__init__(params=params)

        self.alpha = alpha
        self.b1 = beta1
        self.b2 = beta2
        self.gamma = gamma
        self.t = theano.shared(np.float32(1))
        self.eps = eps

        self.ms = [build_shared_zeros(t.shape.eval(), 'm') for t in self.params]
        self.vs = [build_shared_zeros(t.shape.eval(), 'v') for t in self.params]

    def updates(self, loss=None):
        super(Adam, self).updates(loss=loss)
        self.b1_t = self.b1 * self.gamma ** (self.t - 1)

        for m, v, param, gparam \
        in zip(self.ms, self.vs, self.params, self.gparams):
            _m = self.b1_t * m + (1 - self.b1_t) * gparam
            _v = self.b2 * v + (1 - self.b2) * gparam ** 2

            m_hat = _m / (1 - self.b1 ** self.t)
            v_hat = _v / (1 - self.b2 ** self.t)

            self.updates[param] = param - self.alpha*m_hat / (T.sqrt(v_hat) + self.eps)
            self.updates[m] = _m
            self.updates[v] = _v
        self.updates[self.t] = self.t + 1.0

        return self.updates

In [0]:
#Himanshu
class Result(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def negative_log_likelihood(self):
        self.prob_of_y_given_x = T.nnet.softmax(self.x)
        return -T.mean(T.log(self.prob_of_y_given_x)[T.arange(self.y.shape[0]), self.y])

    def cross_entropy(self):
        self.prob_of_y_given_x = T.nnet.softmax(self.x)
        return T.mean(T.nnet.categorical_crossentropy(self.prob_of_y_given_x, self.y))

    def mean_squared_error(self):
        return T.mean((self.x - self.y) ** 2)

    def errors(self):
        if self.y.ndim != self.y_pred.ndim:
            raise TypeError('y should have the same shape as self.y_pred',
                            ('y', self.y.type, 'y_pred', self.y_pred.type))

        if self.y.dtype.startswith('int'):
            self.prob_of_y_given_x = T.nnet.softmax(self.x)
            self.y_pred = T.argmax(self.prob_of_y_given_x, axis=1)
            return T.mean(T.neq(self.y_pred, self.y))
        else:
            return NotImplementedError()

    def accuracy(self):
        if self.y.dtype.startswith('int'):
            self.prob_of_y_given_x = T.nnet.softmax(self.x)
            self.y_pred = T.argmax(self.prob_of_y_given_x, axis=1)
            return T.mean(T.eq(self.y_pred, self.y))
        else:
            return NotImplementedError()

**To be done once everything above this text is finished**

**CharCNN Architecture Contents**

In [0]:
class CharSCNN(object):
    def __init__(self,rng,batchsize=100,activation=relu):
        
        import char_load
        (num_sent, char_cnt, word_cnt, max_word_len, max_sen_len,k_chr, k_wrd, x_chr, x_wrd, y) = char_load.read("tweets_clean.txt")

        dim_word = 30
        dim_char = 5
        cl_word = 300
        cl_char = 50
        k_word = k_wrd
        k_char = k_chr

        data_train_word,data_test_word,data_train_char,data_test_char,target_train,target_test = train_test_split(x_wrd, x_chr, y, random_state=1234, test_size=0.1)

        x_train_word = theano.shared(np.asarray(data_train_word, dtype='int16'), borrow=True)
        x_train_char = theano.shared(np.asarray(data_train_char, dtype='int16'), borrow=True)
        y_train = theano.shared(np.asarray(target_train, dtype='int8'), borrow=True)
        x_test_word = theano.shared(np.asarray(data_test_word, dtype='int16'), borrow=True)
        x_test_char = theano.shared(np.asarray(data_test_char, dtype='int16'), borrow=True)
        y_test = theano.shared(np.asarray(target_test, dtype='int8'), borrow=True)

        self.n_train_batches = x_train_word.get_value(borrow=True).shape[0] / batchsize
        self.n_test_batches = x_test_word.get_value(borrow=True).shape[0] / batchsize        
        
        """symbol definition"""
        index = T.iscalar()
        x_wrd = T.wmatrix('x_wrd')
        x_chr = T.wtensor3('x_chr')
        y = T.bvector('y')
        train = T.iscalar('train')

        """network definition"""
        layer_char_embed_input = x_chr#.reshape((batchsize, max_sen_len, max_word_len))

        layer_char_embed = EmbedIDLayer(rng,layer_char_embed_input,n_input=char_cnt,n_output=dim_char)

        layer1_input = layer_char_embed.output.reshape((batchsize*max_sen_len, 1, max_word_len, dim_char))

        layer1 = ConvolutionalLayer(rng,layer1_input,filter_shape=(cl_char, 1, k_char, dim_char),image_shape=(batchsize*max_sen_len, 1, max_word_len, dim_char))

        layer2 = MaxPoolingLayer(layer1.output,poolsize=(max_word_len-k_char+1, 1))

        layer_word_embed_input = x_wrd #.reshape((batchsize, max_sen_len))

        layer_word_embed = EmbedIDLayer(rng,layer_word_embed_input,n_input=word_cnt,n_output=dim_word)

        layer3_word_input = layer_word_embed.output.reshape((batchsize, 1, max_sen_len, dim_word))
        layer3_char_input = layer2.output.reshape((batchsize, 1, max_sen_len, cl_char))

        layer3_input = T.concatenate([layer3_word_input,layer3_char_input],axis=3)#.reshape((batchsize, 1, max_sen_len, dim_word+cl_char))

        layer3 = ConvolutionalLayer(rng,layer3_input,filter_shape=(cl_word, 1, k_word, dim_word + cl_char),image_shape=(batchsize, 1, max_sen_len, dim_word + cl_char),activation=activation)

        layer4 = MaxPoolingLayer(layer3.output,poolsize=(max_sen_len-k_word+1, 1))

        layer5_input = layer4.output.reshape((batchsize, cl_word))

        layer5 = FullyConnectedLayer(rng,dropout(rng, layer5_input, train),n_input=cl_word,n_output=50,activation=activation)

        layer6_input = layer5.output

        layer6 = FullyConnectedLayer(rng,dropout(rng, layer6_input, train, p=0.1),n_input=50,n_output=2,activation=None)

        result = Result(layer6.output, y)
        loss = result.negative_log_likelihood()
        accuracy = result.accuracy()
        params = layer6.params+layer5.params+layer3.params+layer_word_embed.params+layer1.params+layer_char_embed.params
        updates = RMSprop(learning_rate=0.001, params=params).updates(loss)

        self.train_model = theano.function(inputs=[index],outputs=[loss, accuracy],updates=updates,
            givens={
                x_wrd: x_train_word[index*batchsize: (index+1)*batchsize],
                x_chr: x_train_char[index*batchsize: (index+1)*batchsize],
                y: y_train[index*batchsize: (index+1)*batchsize],
                train: np.cast['int32'](1)
            }
        )

        self.test_model = theano.function(inputs=[index],outputs=[loss, accuracy],
            givens={
                x_wrd: x_test_word[index*batchsize: (index+1)*batchsize],
                x_chr: x_test_char[index*batchsize: (index+1)*batchsize],
                y: y_test[index*batchsize: (index+1)*batchsize],
                train: np.cast['int32'](0)
            }
        )

#Indraneel
    def train_and_test(self, n_epoch=100):
        epoch = 0
        accuracies = []
        while epoch < n_epoch:
            epoch += 1
            sum_loss = 0
            sum_accuracy = 0
            for batch_index in xrange(self.n_train_batches):
                batch_loss, batch_accuracy = self.train_model(batch_index)
                sum_loss = 0
                sum_accuracy = 0
                for batch_index in xrange(self.n_test_batches):
                    batch_loss, batch_accuracy = self.test_model(batch_index)
                    sum_loss += batch_loss
                    sum_accuracy += batch_accuracy
                loss = sum_loss / self.n_test_batches
                accuracy = sum_accuracy / self.n_test_batches
                accuracies.append(accuracy)

                print('epoch: {}, test mean loss={}, test accuracy={}'.format(epoch, loss, accuracy))
                print('')
        return accuracies



In [0]:
#Rough work for Keras model
model = Sequential()
model.add(Embedding(char_cnt,dim_char,input_length=char_cnt))#Input embedding char embedding
model.add(Conv2D(filters=c1_char, kernel_size=(k_char,dim_char),input_shape=(batchsize*max_sen_len, max_word_len, dim_char))) 
#https://stackoverflow.com/questions/30633181/the-output-size-of-theano-tensor-nnet-conv-conv2d











In [0]:
ConvolutionalLayer(rng,layer1_input,filter_shape=(cl_char, 1, k_char, dim_char),image_shape=(batchsize*max_sen_len, 1, max_word_len, dim_char))


**Work Left : Main function and Visualization Functions(Using Matplotlib)**

In [0]:
if __name__ == '__main__':
    random_state = 1234
    rng = np.random.RandomState(random_state)
    charscnn = CharSCNN(rng, batchsize=10, activation=relu)
    charscnn.train_and_test(n_epoch=3)


